<a href="https://colab.research.google.com/github/AhmedBenAbdessalam/MNIST_simple_model/blob/main/MNIST_project_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [65]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

##Preprocessing the data

In [66]:
BUFFER_SIZE = 70000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [67]:
mnist_dataset , mnist_info = tfds.load(name="mnist",with_info=True, as_supervised=True)

In [68]:
mnist_train, mnist_test = mnist_dataset['train'] , mnist_dataset['test']

In [69]:
def grayscale_normalizer(image,label):
  image = tf.cast(image,tf.float32)
  image /= 255.
  return image,label

In [70]:
scaled_mnist_train, scaled_mnist_test = mnist_train.map(grayscale_normalizer), mnist_train.map(grayscale_normalizer)  

In [71]:
num_validation_data = .1 * mnist_info.splits['train'].num_examples
num_validation_data = tf.cast(num_validation_data , tf.int64)
num_test_data = mnist_info.splits['test'].num_examples
num_test_data = tf.cast(num_test_data,tf.int64)

In [72]:
scaled_mnist_train.shuffle(BUFFER_SIZE)

<ShuffleDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [73]:
train_data = scaled_mnist_train.skip(num_validation_data)
validation_data = scaled_mnist_train.take(num_validation_data)
test_data = scaled_mnist_test

In [74]:
train_data = train_data.batch(BATCH_SIZE)
validation_data= validation_data.batch(num_validation_data)
test_data = test_data.batch(num_train_data)

##Model

In [75]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(filters=50,kernel_size=5,activation='relu' , input_shape=(28,28,1)),
                             tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                             tf.keras.layers.Conv2D(filters=50,kernel_size=3,activation='relu'),
                             tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10)
])

In [76]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 50)        1300      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 50)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 50)        22550     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 50)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1250)              0         
                                                                 
 dense_1 (Dense)             (None, 10)               

In [77]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [78]:
model.compile(optimizer="adam" , loss=loss_fn , metrics=['accuracy'])

In [79]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor ='val_loss',
    mode='auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights= True
)

In [80]:
model.fit(
    train_data,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
     verbose=2,
     callbacks=[early_stopping],
     validation_data=validation_data)

Epoch 1/20
422/422 - 63s - loss: 0.2682 - accuracy: 0.9229 - val_loss: 0.0918 - val_accuracy: 0.9735 - 63s/epoch - 149ms/step
Epoch 2/20
422/422 - 59s - loss: 0.0721 - accuracy: 0.9776 - val_loss: 0.0674 - val_accuracy: 0.9810 - 59s/epoch - 139ms/step
Epoch 3/20
422/422 - 59s - loss: 0.0516 - accuracy: 0.9841 - val_loss: 0.0605 - val_accuracy: 0.9827 - 59s/epoch - 139ms/step
Epoch 4/20
422/422 - 59s - loss: 0.0413 - accuracy: 0.9874 - val_loss: 0.0545 - val_accuracy: 0.9840 - 59s/epoch - 139ms/step
Epoch 5/20
422/422 - 59s - loss: 0.0346 - accuracy: 0.9895 - val_loss: 0.0507 - val_accuracy: 0.9855 - 59s/epoch - 139ms/step
Epoch 6/20
422/422 - 59s - loss: 0.0292 - accuracy: 0.9912 - val_loss: 0.0484 - val_accuracy: 0.9858 - 59s/epoch - 139ms/step
Epoch 7/20
422/422 - 59s - loss: 0.0251 - accuracy: 0.9928 - val_loss: 0.0474 - val_accuracy: 0.9865 - 59s/epoch - 139ms/step
Epoch 8/20
422/422 - 58s - loss: 0.0214 - accuracy: 0.9939 - val_loss: 0.0472 - val_accuracy: 0.9875 - 58s/epoch - 139

##Testing

In [83]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

2/2 [==============================] - 15s 1s/step - loss: 0.0203 - accuracy: 0.9937
Test loss: 0.0203. Test accuracy: 99.37%
